### Задание №6

Задание состоит в том, чтобы реализовать базовые алгоритмы линейной алгебры с помощью базовых средств объектно-ориентированного программирования Python.

Для решения задачи требуется реализовать класс Matrix с базовыми алгоритмами линейной алгебры:

\_\_mul\_\_ - умножение матриц

\_\_add\_\_ - поэлементное сложение матриц

\_\_sub\_\_ - поэлементное вычитание матриц

\_\_pow\_\_ - возведение элементов матрицы в скаларную степень

transpose() - транспонирование матрицы

inverse() - обращение матрицы

det() - определитель матрицы

shape() - размер матрицы (кортеж)

sum() - сумма всех элементов матрицы

а также служебные методы:

\_\_getitem\_\_ - получить элемент по индексу

\_\_setitem\_\_ - задать элемент по индексу

\_\_repr\_\_ и \_\_str\_\_

### Отправка задания

Для сдачи задания необходимо отправить боту @py2022sharebot с указанием seminar06_1 два файла:
1. result.json (файл, создаваемый в последней ячейке)
2. seminar06_1.ipynb (этот ноутбук)

Автоматическая проверка отправки будет реализована командой /check seminar06_1.

In [1]:
import sys
import os
import math
import copy
import json
import random

Требуется реализовать методы в следующем классе, отмеченные #TODO:

In [19]:
class Matrix:
    def __init__(self, nrows, ncols, init="zeros"):
        """Конструктор класса Matrix.
        Создаёт матрицу резмера nrows x ncols и инициализирует её методом init.
        nrows - количество строк матрицы
        ncols - количество столбцов матрицы
        init - метод инициализации элементов матрицы:
            "zeros" - инициализация нулями
            "ones" - инициализация единицами
            "random" - случайная инициализация
            "eye" - матрица с единицами на главной диагонали
        """
        if nrows < 0 or ncols < 0:
            raise ValueError("nrows and ncols must be non-negative")
        if init not in ["zeros", "ones", "random", "eye"]:
            raise ValueError("Invalid initialization method")
        self.nrows = nrows
        self.ncols = ncols
        self.data = []
        if init == "zeros":
            self.data = [[0 for _ in range(self.ncols)] for _ in range(self.nrows)]
        elif init == "ones":
            self.data = [[1 for _ in range(self.ncols)] for _ in range(self.nrows)]
        elif init == "random":
            self.data = [[random.random() for _ in range(self.ncols)] for _ in range(self.nrows)]
        elif init == "eye":
            self.data = [[1 if i == j else 0 for j in range(self.ncols)] for i in range(self.nrows)]
    
    @staticmethod
    def from_dict(data):
        "Десериализация матрицы из словаря"
        ncols = data["ncols"]
        nrows = data["nrows"]
        items = data["data"]
        assert len(items) == ncols*nrows
        result = Matrix(nrows, ncols)
        for row in range(nrows):
            for col in range(ncols):
                result[(row, col)] = items[ncols*row + col]
        return result
    
    @staticmethod
    def to_dict(matr):
        "Сериализация матрицы в словарь"
        assert isinstance(matr, Matrix)
        nrows, ncols = matr.shape()
        data = []
        for row in range(nrows):
            for col in range(ncols):
                data.append(matr[(row, col)])
        return {"nrows": nrows, "ncols": ncols, "data": data}
    
    def __str__(self):
        res = ""
        for row in self.data:
            res += " ".join(str(element) for element in row) + "\n"
        return res
    
    def __repr__(self):
        return f"Matrix({self.nrows}, {self.ncols}, 'custom')"
    
    def shape(self):
        "Вернуть кортеж размера матрицы (nrows, ncols)"
        return (self.nrows, self.ncols)
    
    def __getitem__(self, index):
        """Получить элемент матрицы по индексу index
        index - список или кортеж, содержащий два элемента
        """
        if not isinstance(index, (tuple, list)) or len(index) != 2:
            raise ValueError("Index must be a list or tuple with two elements")
        row, col = index
        if row < 0 or row >= self.nrows or col < 0 or col >= self.ncols:
            raise IndexError("Index out of bounds")
        return self.data[row][col]
    
    def __setitem__(self, index, value):
        """Задать элемент матрицы по индексу index
        index - список или кортеж, содержащий два элемента
        value - Устанавливаемое значение
        """
        if not isinstance(index, (tuple, list)) or len(index) != 2:
            raise ValueError("Index must be a list or tuple with two elements")
        row, col = index
        if row < 0 or row >= self.nrows or col < 0 or col >= self.ncols:
            raise IndexError("Index out of bounds")
        self.data[row][col] = value
    
    def __sub__(self, rhs):
        "Вычесть матрицу rhs и вернуть результат"
        if self.nrows != rhs.nrows or self.ncols != rhs.ncols:
            raise ValueError("Matrix sizes do not match for subtraction")
        result = Matrix(self.nrows, self.ncols)
        for row in range(self.nrows):
            for col in range(self.ncols):
                result[(row, col)] = self[(row, col)] - rhs[(row, col)]
        return result
    
    def __add__(self, rhs):
        "Сложить с матрицей rhs и вернуть результат"
        if self.nrows != rhs.nrows or self.ncols != rhs.ncols:
            raise ValueError("Matrix sizes do not match for addition")
        result = Matrix(self.nrows, self.ncols)
        for row in range(self.nrows):
            for col in range(self.ncols):
                result[(row, col)] = self[(row, col)] + rhs[(row, col)]
        return result
    
    def __mul__(self, rhs):
        "Умножить на матрицу rhs и вернуть результат"
        if self.ncols != rhs.nrows:
            raise ValueError("Matrix sizes do not match for multiplication")
        result = Matrix(self.nrows, rhs.ncols)
        for i in range(self.nrows):
            for j in range(rhs.ncols):
                dot_product = 0
                for k in range(self.ncols):
                    dot_product += self[(i, k)] * rhs[(k, j)]
                result[(i, j)] = dot_product
        return result
    
    def __pow__(self, power):
        "Возвести все элементы в степень power и вернуть результат"
        result = Matrix(self.nrows, self.ncols)
        for row in range(self.nrows):
            for col in range(self.ncols):
                result[(row, col)] = self[(row, col)] ** power
        return result
    
    def sum(self):
        "Вернуть сумму всех элементов матрицы"
        total = 0
        for row in range(self.nrows):
            for col in range(self.ncols):
                total += self[(row, col)]
        return total
        
    def det(self):
        "Вычислить определитель матрицы"
        if self.nrows != self.ncols:
            raise ValueError("Matrix must be square to compute the determinant")
        det_value = 1.0
        mat = [row[:] for row in self.data]
        for col in range(self.ncols):
            pivot_row = col
            while pivot_row < self.nrows and mat[pivot_row][col] == 0:
                pivot_row += 1
            if pivot_row == self.nrows:
                return 0.0
            if pivot_row != col:
                mat[pivot_row], mat[col] = mat[col], mat[pivot_row]
                det_value *= -1
            pivot_element = mat[col][col]
            det_value *= pivot_element
            for j in range(col, self.ncols):
                mat[col][j] /= pivot_element
            for i in range(col + 1, self.nrows):
                factor = mat[i][col]
                for j in range(col, self.ncols):
                    mat[i][j] -= factor * mat[col][j]
        for i in range(self.nrows):
            det_value *= mat[i][i]
        return det_value
    
    def transpose(self):
        "Транспонировать матрицу и вернуть результат"
        result = Matrix(self.ncols, self.nrows)
        for row in range(self.nrows):
            for col in range(self.ncols):
                result[(col, row)] = self[(row, col)]
        return result

    def submatrix(self, row, col):
        submat = Matrix(self.nrows - 1, self.ncols - 1, init="zeros")
        i_offset = 0
        for i in range(self.nrows):
            if i == row:
                i_offset = 1
                continue
            j_offset = 0
            for j in range(self.ncols):
                if j == col:
                    j_offset = 1
                    continue
                submat[(i - i_offset, j - j_offset)] = self[(i, j)]
        return submat
    
    def inv(self):
        "Вычислить обратную матрицу и вернуть результат"
        if self.nrows != self.ncols:
            raise ArithmeticError("Matrix must be square to compute its inverse")
        det = self.det()
        if det == 0:
            raise ArithmeticError("Matrix is singular; its inverse does not exist")
        minors = Matrix(self.nrows, self.ncols, init="zeros")
        for i in range(self.nrows):
            for j in range(self.ncols):
                submat = self.submatrix(i, j)
                minors[(i, j)] = submat.det() * ((-1) ** (i + j))
        minors = minors.transpose()
        inv_matrix = Matrix(minors.nrows, minors.ncols)
        for row in range(minors.nrows):
            for col in range(minors.ncols):
                inv_matrix[(row, col)] = minors[(row, col)] * (1 / det)
        return inv_matrix

    def tonumpy(self):
        "Приведение к массиву numpy"
        numpy_array = np.zeros((self.nrows, self.ncols))
        for row in range(self.nrows):
            for col in range(self.ncols):
                numpy_array[row, col] = self[(row, col)]
        return numpy_array

In [20]:
def load_file(filename):
    with open(filename, "r") as f:
        input_file = json.load(f)
        A = Matrix.from_dict(input_file["A"])
        B = Matrix.from_dict(input_file["B"])
    return A, B

Проверка реализованных методов

In [21]:
# Задайте в filename путь к файлу, полученному от бота
filename = "input_021.json"
A, B = load_file(filename)
print("Матрица A: ")
print(A)
print("Матрица B: ")
print(B)
C = A*B
print("Матрица C = A*B: ")
print(C)
C_t = C.transpose()
print("Транспонированная матрица C: ")
print(C_t)
C_inv = C.inv()
print("Матрица, обратная C: ")
print(C_inv)
E = Matrix(C_inv.ncols, C_inv.nrows, init="eye")
D = C_inv + E
print("Матрица D равная сумме C_inv и единичной матрицы: ")
print(D)
D_det = D.det()
print("Определитель матрицы D: ", D_det)
D_norm = (D**2).sum()**0.5
print("Норма Фробениуса матрицы D: ", D_norm)

Матрица A: 
0.8923407983171103 0.9727476699471198 0.443223447356223 0.9054686343370189
0.9044227961911576 0.1916389251396191 0.4541714887452599 0.7032196096037262
0.9338494492951434 0.7810459200489549 0.8863989944619759 0.5196847307098503

Матрица B: 
0.8699495372097933 0.8179794418591091 0.7635558302750379
0.9596854437927368 0.8937833882653984 0.5496784800903712
0.7435723965830868 0.5423290032244916 0.4996412704030525
0.14308631784880999 0.9785054359899753 0.30157474128370243

Матрица C = A*B: 
2.168952137648299 2.725721047755765 1.7105696753254718
1.4090457668323595 1.8454975233441202 1.2349131836117062
2.2954218958430705 2.451189737253428 1.741975633711619

Транспонированная матрица C: 
2.168952137648299 1.4090457668323595 2.2954218958430705
2.725721047755765 1.8454975233441202 2.451189737253428
1.7105696753254718 1.2349131836117062 1.741975633711619

Матрица, обратная C: 
1.7856489603896197 -5.278917030912619 1.9888494461116943
3.614207239520273 -1.409247967152628 -2.55000949765624

Сохранение результатов в файл. Не изменяйте этот код. Отправтье файл result.json в ответ на это задание.

In [22]:
A_dict = Matrix.to_dict(A)
B_dict = Matrix.to_dict(B)
C_dict = Matrix.to_dict(C)
Ct_dict = Matrix.to_dict(C_t)
Cinv_dict = Matrix.to_dict(C_inv)
result = {
    "A": A_dict,
    "B": B_dict,
    "C": C_dict,
    "Ct": Ct_dict,
    "Cinv": Cinv_dict,
    "D_det": D_det,
    "D_norm": D_norm
}

In [23]:
def save_file(filename, data):
    with open(filename, "w") as f:
        input_file = json.dump(data, f)

In [24]:
save_file("result.json", result)